In [10]:
import pandas as pd
import requests
import uuid
import re
import datetime 
from datetime import date
from bs4 import BeautifulSoup

def get_links_legislaturas(legislaturas):
    """
    Parametros:
    -----------
    legislatura: list of int numbers of legislatura

    Returns:
    --------
    lista de diccionario con los a el htmls de los debates legislativos
    """
    all_data = []

    for legislatura in legislaturas:
        link_debates = 'http://cronica.diputados.gob.mx/DDebates/' + str(legislatura) + '/index.html'

        r = requests.get(link_debates)
        soup = BeautifulSoup(r.content, 'html.parser')

        links_sesiones = []
        type_sesion = []
        urls = []
        for h in soup.findAll('a'):
            links_sesiones.append('http://cronica.diputados.gob.mx/DDebates/' + str(legislatura) + '/' + h.get('href'))
            type_sesion.append(re.sub('\xa0', '', h.text))
            urls.append(h.get('href'))


        for x, y, urls in zip(links_sesiones, type_sesion, urls):
            r = requests.get(x)
            soup = BeautifulSoup(r.content, 'html.parser')

            for i in soup.findAll('a'):
                year_session = re.findall('[0-9]{4}', soup.findAll('tbody')[1].find('tr').text)[0]
                link_ = 'http://cronica.diputados.gob.mx/DDebates/' + str(legislatura)+'/'+ re.sub('index[.]html', '', urls) +i.get('href')

                fecha_speech = get_date_speech(year_session, link_, i.text)

                temp = {}
                temp['id_debate'] = str(uuid.uuid3(uuid.NAMESPACE_URL,link_))
                temp['legislatura'] = legislatura
                temp['date_debate'] = fecha_speech
                temp['type_sesion'] = y
                temp['link_debate'] = link_
                all_data.append(temp)
    return(all_data)

def get_date_speech(year_session, link_speech, text_day):
    day_speech = re.findall('[0-9]+', text_day)[0]
    month_speech = month_to_number(link_speech.split('/')[-2:][0])
    return(datetime.date(int(year_session), int(month_speech), int(day_speech)))

def month_to_number(m):
    if m == 'ene':
        return(1)
    elif m == 'feb':
        return(2)
    elif m == 'mar':
        return(3)
    elif m == 'abr':
        return(4)
    elif m == 'may':
        return(5)
    elif m == 'jun':
        return(6)
    elif m == 'jul':
        return(7)
    elif m == 'ago':
        return(8)
    elif m == 'sep':
        return(9)
    elif m == 'oct':
        return(10)
    elif m == 'nov':
        return(11)
    elif m == 'dic':
        return(12)

In [11]:
data = get_links_legislaturas([63, 64])

In [17]:
df_data = pd.DataFrame(data)

In [16]:
## get 

In [128]:
def obtain_texto_debate(link):
    r = requests.get(link)
    soup = BeautifulSoup(r.content, 'html.parser')
    contenido_debate = soup.findAll("div", {"class": "Contenido"})[0].text
    
    dict_contenidos = {}
    for i in contenido_debate.split('\n\n\n \n'):
        try:
            name_tema = re.sub('\n', '', re.findall('^[A-Z].*\n',i)[0])
            text_all = re.sub(name_tema, '', i)
            text_all = re.sub('[ ]+$', '', re.sub('^[ ]+', '', re.sub('[ ]+', ' ', re.sub('\n+', ' ', text_all))))
            dict_contenidos[name_tema] = text_all
        except:
            continue
    dict_contenidos['id_debate'] = str(uuid.uuid3(uuid.NAMESPACE_URL,link))
    #main_text = []
    #for x in dict_contenidos.values():
    #    main_text.append(x)
    #return(re.sub('\n', ' ', ' '.join(main_text)))
    print(len(dict_contenidos.keys()))
    return(dict_contenidos)


def obtain_text_sesion(link):
    r = requests.get(link)
    soup = BeautifulSoup(r.content, 'html.parser')
    
    try:
        links_volumenes = []
        for h in soup.findAll('a'):
            if any(re.findall('Volúmen', h.text)):
                links_volumenes.append(h.get('href'))
        
        real_link = '/'.join(link.split('/')[:-1]) + '/' + links_volumenes[0]
        
        texto_debate = obtain_texto_debate(real_link)
        return(texto_debate)
    except:
        return(obtain_texto_debate(link))

In [129]:
for i in df_data.link_debate:
    try:
        obtain_text_sesion(i)
    except:
        print(i)

13
10
27
18
13
11
21
33
28
34
11
26
29
30
23
21
18
20
42
11
31
29
31
31
42
13
22
52
22
61
35
21
43
21
19
139
7
8
20
19
35
49
40
7
36
44
23
27
35
30
27
26
42
31
33
22
30
36
30
34
38
24
39
66
3
31
22
26
33
60
26
34
213
7
13
8
5
14
8
5
10
10
31
8
35
36
32
7
19
23
56
27
27
41
30
44
32
14
24
14
26
31
32
34
17
7
32
44
7
47
51
29
16
44
32
5
8
33
23
25
36
60
7
8
31
28
36
34
44
38
18
29
22
33
22
7
33
33
42
26
29
34
25
25
19
28
38
23
21
62
57
33
5
8
8
33
14
26
8
8
26
25
25
19
7
26
21
21
11
http://cronica.diputados.gob.mx/DDebates/63/3er/1P/Ord/oct/Seseion18.html
21
27
24
35
12
31
19
28
32
18
23
10
30
46
62
29
54
5
8
17
23
38
62
78
9
6
http://cronica.diputados.gob.mx/DDebates/63/3er/2P/Ord/feb/Sesion2.html
28
9
34
22
29
33
30
36
25
22
38
29
19
23
27
24
23
43
23
39
35
29
17
53
75
16
12
13
31
21
25
21
18
31
34
18
36
8
19
6
12
54
4
13
7
38
5
13
5
22
39
32
17
3
41
44
15
23
5
9
28
43
14
35
11
28
6
10
21
8
20
25
20
48
7
6
http://cronica.diputados.gob.mx/DDebates/64/1er/1P/1Extra/ene/01L64A1E1022.html
6